# Kapitola 4: AI vs. Datová věda - Kdo je kdo?

---

## Co se naučíte

V této kapitole si prakticky vyzkoušíte:

1. **Roli datového vědce** - průzkum a vizualizace dat
2. **Roli statistika** - základní statistická analýza
3. **Roli AI inženýra** - stavba prediktivního modelu
4. **Spolupráci všech rolí** na slavném Iris datasetu

---

## Spuštění v Google Colab

Tento notebook funguje přímo v bezplatném Google Colab - stačí kliknout na tlačítko "Open in Colab" nebo nahrát tento soubor do Colab.

## Část 1: Úvod - Tým snů pro AI

### Analogie

Představte si, že stavíte autonomní auto:

| Role | Co dělá | V AI světě |
|------|---------|------------|
| **Datový vědec** | Analyzuje dopravní data | Hledá vzory v datech |
| **Statistik** | Ověřuje spolehlivost měření | Matematicky dokazuje vztahy |
| **AI inženýr** | Staví rozhodovací systém | Vytváří prediktivní model |
| **Kognitivní vědec** | Studuje lidské chování řidičů | Inspirace pro algoritmy |
| **Filozof** | Řeší etické otázky | Morální kompas AI |

## Část 2: Instalace knihoven a načtení dat

Použijeme slavný **Iris dataset** - obsahuje měření 150 kosatců tří různých druhů.

In [ ]:
# Instalace knihoven (v Colabu už jsou předinstalované)
# !pip install pandas seaborn scikit-learn matplotlib

# Import knihoven
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Nastavení vizualizací
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Knihovny úspěšně načteny!")

In [ ]:
# Načtení Iris datasetu
iris = load_iris()

# Vytvoření DataFrame pro snadnější práci
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['druh'] = [iris.target_names[i] for i in iris.target]

print("📊 Iris dataset načten!")
print(f"   Počet vzorků: {len(df)}")
print(f"   Počet druhů: {df['druh'].nunique()}")
print(f"   Druhy: {list(df['druh'].unique())}")

## Část 3: Role Datového vědce - Průzkum dat

### Cíl datového vědce: **Pochopit data, najít vzory, vytvořit vhled**

Datový vědec je jako detektiv - hledá stopy v datech a sestavuje příběh.

In [ ]:
# 3.1: První pohled na data
print("🔍 DATOVÝ VĚDEC: Podívám se na strukturu dat\n")

print("Prvních 5 řádků:")
df.head()

In [ ]:
# 3.2: Základní informace o datech
print("📋 Informace o datasetu:\n")
print(df.info())

print("\n" + "="*50)
print("\n📈 Počet vzorků každého druhu:")
print(df['druh'].value_counts())

In [ ]:
# 3.3: Vizualizace - pairplot
print("🎨 DATOVÝ VĚDEC: Vytvořím vizualizaci vztahů mezi proměnnými\n")

# Pairplot ukazuje vztahy mezi všemi páry proměnných
sns.pairplot(df, hue='druh', palette='husl', height=2.5)
plt.suptitle('Vztahy mezi měřeními pro tři druhy kosatců', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

print("\n💡 VHLED: Setosa (modrá) je jasně odlišitelná od ostatních druhů!")

In [ ]:
# 3.4: Boxplot pro porovnání druhů
print("📊 DATOVÝ VĚDEC: Porovnám rozložení měření pro každý druh\n")

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Porovnání měření pro tři druhy kosatců', fontsize=14)

vlastnosti = ['sepal length (cm)', 'sepal width (cm)', 
              'petal length (cm)', 'petal width (cm)']
nazvy_cz = ['Délka kalichu', 'Šířka kalichu', 
            'Délka okvětí', 'Šířka okvětí']

for idx, (vlastnost, nazev) in enumerate(zip(vlastnosti, nazvy_cz)):
    ax = axes[idx // 2, idx % 2]
    sns.boxplot(x='druh', y=vlastnost, data=df, ax=ax, palette='husl')
    ax.set_title(nazev)
    ax.set_xlabel('Druh')
    ax.set_ylabel('Hodnota (cm)')

plt.tight_layout()
plt.show()

print("💡 VHLED: Délka a šířka okvětí nejlépe rozlišují druhy!")

### Shrnutí role datového vědce

✅ Načetl a prozkoumal data  
✅ Vytvořil vizualizace pro pochopení vztahů  
✅ Identifikoval klíčový poznatek: **Okvětní lístky nejlépe rozlišují druhy**

**Hlavní produkt datového vědce: VHLED do dat**

## Část 4: Role Statistika - Ověření poznatků

### Cíl statistika: **Matematicky ověřit a kvantifikovat vztahy**

Statistik je jako soudce - ověřuje, zda jsou poznatky pravdivé a spolehlivé.

In [ ]:
# 4.1: Základní statistiky
print("📐 STATISTIK: Spočítám základní statistiky pro každý druh\n")

# Statistiky pro délku okvětí podle druhu
print("Délka okvětního lístku (petal length) - statistiky:")
print("="*50)

statistiky = df.groupby('druh')['petal length (cm)'].agg([
    ('průměr', 'mean'),
    ('medián', 'median'),
    ('min', 'min'),
    ('max', 'max'),
    ('směr_odch', 'std')
]).round(2)

print(statistiky)

In [ ]:
# 4.2: Korelační analýza
print("📊 STATISTIK: Spočítám korelace mezi proměnnými\n")

# Korelační matice (bez sloupce 'druh')
korelace = df.drop('druh', axis=1).corr()

# Vizualizace korelační matice
plt.figure(figsize=(8, 6))
sns.heatmap(korelace, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', linewidths=0.5)
plt.title('Korelační matice - vztahy mezi měřeními')
plt.tight_layout()
plt.show()

print("\n💡 INTERPRETACE:")
print("   - Délka a šířka okvětí jsou silně korelované (0.96)")
print("   - Délka kalichu a okvětí jsou také korelované (0.87)")

In [ ]:
# 4.3: Statistický test - jsou průměry různé?
print("🧪 STATISTIK: Ověřím, zda jsou rozdíly mezi druhy statisticky významné\n")

from scipy import stats

# ANOVA test pro délku okvětí
setosa = df[df['druh'] == 'setosa']['petal length (cm)']
versicolor = df[df['druh'] == 'versicolor']['petal length (cm)']
virginica = df[df['druh'] == 'virginica']['petal length (cm)']

f_stat, p_value = stats.f_oneway(setosa, versicolor, virginica)

print(f"ANOVA test pro délku okvětí:")
print(f"   F-statistika: {f_stat:.2f}")
print(f"   p-hodnota: {p_value:.2e}")

if p_value < 0.05:
    print("\n✅ ZÁVĚR: Rozdíly mezi druhy JSOU statisticky významné!")
    print("   (S 95% jistotou můžeme říct, že průměry se liší)")
else:
    print("\n❌ ZÁVĚR: Rozdíly NEJSOU statisticky významné")

### Shrnutí role statistika

✅ Spočítal základní statistiky (průměr, medián, odchylka)  
✅ Analyzoval korelace mezi proměnnými  
✅ Provedl statistický test pro ověření významnosti  

**Hlavní produkt statistika: DŮKAZ a INTERPRETACE**

## Část 5: Role AI Inženýra - Stavba modelu

### Cíl AI inženýra: **Vytvořit systém, který automaticky klasifikuje nové vzorky**

AI inženýr je jako stavitel - bere poznatky a staví z nich fungující stroj.

In [ ]:
# 5.1: Příprava dat pro model
print("🔧 AI INŽENÝR: Připravím data pro trénování modelu\n")

# Rozdělení dat na features (X) a target (y)
X = iris.data  # Měření (4 sloupce)
y = iris.target  # Druhy (0, 1, 2)

# Rozdělení na trénovací a testovací sadu (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"📊 Rozdělení dat:")
print(f"   Trénovací sada: {len(X_train)} vzorků")
print(f"   Testovací sada: {len(X_test)} vzorků")

In [ ]:
# 5.2: Vytvoření a trénování modelu
print("🤖 AI INŽENÝR: Vytvořím a natrénuji model K-nejbližších sousedů (KNN)\n")

# Vytvoření KNN modelu (3 nejbližší sousedé)
model = KNeighborsClassifier(n_neighbors=3)

# Trénování modelu
model.fit(X_train, y_train)

print("✅ Model natrénován!")
print("\n📚 Jak KNN funguje:")
print("   1. Dostane nový vzorek k klasifikaci")
print("   2. Najde 3 nejpodobnější vzorky z trénovacích dat")
print("   3. Určí druh podle většiny těchto sousedů")

In [ ]:
# 5.3: Vyhodnocení modelu
print("📈 AI INŽENÝR: Vyhodnotím přesnost modelu na testovacích datech\n")

# Predikce na testovací sadě
y_pred = model.predict(X_test)

# Výpočet přesnosti
presnost = accuracy_score(y_test, y_pred)

print(f"🎯 Přesnost modelu: {presnost * 100:.1f}%")
print(f"   (Z {len(y_test)} testovacích vzorků model správně klasifikoval {int(presnost * len(y_test))})") 

In [ ]:
# 5.4: Matice záměn (Confusion Matrix)
print("📊 AI INŽENÝR: Vytvořím matici záměn pro detailní analýzu\n")

# Výpočet matice záměn
cm = confusion_matrix(y_test, y_pred)

# Vizualizace
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.xlabel('Predikovaný druh')
plt.ylabel('Skutečný druh')
plt.title('Matice záměn - Jak si model vedl?')
plt.tight_layout()
plt.show()

print("💡 INTERPRETACE:")
print("   Čísla na diagonále = správné klasifikace")
print("   Čísla mimo diagonálu = chyby")

In [ ]:
# 5.5: Klasifikace nového vzorku
print("🌸 AI INŽENÝR: Vyzkoušíme model na novém vzorku\n")

# Nový, neznámý vzorek (měření v cm)
novy_vzorek = [[5.1, 3.5, 1.4, 0.2]]  # Typický setosa

print(f"Nový vzorek:")
print(f"   Délka kalichu: {novy_vzorek[0][0]} cm")
print(f"   Šířka kalichu: {novy_vzorek[0][1]} cm")
print(f"   Délka okvětí: {novy_vzorek[0][2]} cm")
print(f"   Šířka okvětí: {novy_vzorek[0][3]} cm")

# Predikce
predikce = model.predict(novy_vzorek)
druh = iris.target_names[predikce[0]]

print(f"\n🎯 Model predikuje: {druh.upper()}")

In [ ]:
# 5.6: Interaktivní klasifikace
print("🎮 AI INŽENÝR: Vytvořím interaktivní funkci pro klasifikaci\n")

def klasifikuj_kosatec(delka_kalichu, sirka_kalichu, delka_okveti, sirka_okveti):
    """
    Klasifikuje kosatec na základě jeho měření.
    
    Parametry:
        delka_kalichu: Délka kališního lístku v cm
        sirka_kalichu: Šířka kališního lístku v cm
        delka_okveti: Délka okvětního lístku v cm
        sirka_okveti: Šířka okvětního lístku v cm
    
    Vrací:
        Název druhu kosatce
    """
    vzorek = [[delka_kalichu, sirka_kalichu, delka_okveti, sirka_okveti]]
    predikce = model.predict(vzorek)
    return iris.target_names[predikce[0]]

# Test funkce
print("Test funkce klasifikuj_kosatec():")
print(f"   [5.1, 3.5, 1.4, 0.2] → {klasifikuj_kosatec(5.1, 3.5, 1.4, 0.2)}")
print(f"   [6.0, 2.9, 4.5, 1.5] → {klasifikuj_kosatec(6.0, 2.9, 4.5, 1.5)}")
print(f"   [6.5, 3.0, 5.5, 2.0] → {klasifikuj_kosatec(6.5, 3.0, 5.5, 2.0)}")

### Shrnutí role AI inženýra

✅ Připravil data pro strojové učení  
✅ Vytvořil a natrénoval klasifikační model  
✅ Vyhodnotil přesnost modelu  
✅ Vytvořil funkci pro automatickou klasifikaci  

**Hlavní produkt AI inženýra: AUTOMATIZOVANÁ AKCE**

## Část 6: Porovnání rolí

### AI vs. Datová věda vs. Statistika

In [ ]:
# Vizuální porovnání rolí
print("📊 POROVNÁNÍ ROLÍ V DATA/AI TÝMU\n")
print("="*60)

porovnani = {
    'Role': ['Datový vědec', 'Statistik', 'AI inženýr'],
    'Hlavní cíl': ['Pochopit data', 'Ověřit hypotézy', 'Vytvořit predikce'],
    'Nástroje': ['pandas, seaborn', 'scipy, statsmodels', 'sklearn, TensorFlow'],
    'Výstup': ['Grafy, reporty', 'p-hodnoty, důkazy', 'Model, API'],
    'Otázka': ['Co se děje?', 'Je to pravda?', 'Co bude?']
}

df_porovnani = pd.DataFrame(porovnani)
print(df_porovnani.to_string(index=False))

In [ ]:
# Vizualizace přístupu jednotlivých rolí
print("\n🎨 Jak každá role přistupuje k datům:\n")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Datový vědec - průzkum
axes[0].scatter(df['petal length (cm)'], df['petal width (cm)'], 
                c=iris.target, cmap='viridis', alpha=0.7)
axes[0].set_xlabel('Délka okvětí')
axes[0].set_ylabel('Šířka okvětí')
axes[0].set_title('DATOVÝ VĚDEC\n"Jaké vzory vidím?"')

# 2. Statistik - analýza
prumery = df.groupby('druh')['petal length (cm)'].mean()
axes[1].bar(prumery.index, prumery.values, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1].set_ylabel('Průměrná délka okvětí (cm)')
axes[1].set_title('STATISTIK\n"Jaké jsou průměry?"')
axes[1].axhline(y=prumery.mean(), color='red', linestyle='--', label='Celkový průměr')
axes[1].legend()

# 3. AI inženýr - model
from sklearn.inspection import DecisionBoundaryDisplay
# Jednoduchá vizualizace rozhodovacích hranic
xx, yy = np.meshgrid(np.linspace(0, 7, 100), np.linspace(0, 3, 100))
# Pro zjednodušení použijeme jen 2 features
simple_model = KNeighborsClassifier(n_neighbors=3)
simple_model.fit(X_train[:, 2:4], y_train)  # jen petal length a width
Z = simple_model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
axes[2].contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
axes[2].scatter(df['petal length (cm)'], df['petal width (cm)'], 
                c=iris.target, cmap='viridis', edgecolors='black')
axes[2].set_xlabel('Délka okvětí')
axes[2].set_ylabel('Šířka okvětí')
axes[2].set_title('AI INŽENÝR\n"Kam patří nový vzorek?"')

plt.tight_layout()
plt.show()

## Část 7: Mini-projekt - Prozkoumejte vlastní data

### Úkol: Vyzkoušejte si všechny tři role

In [ ]:
# MINI-PROJEKT: Použijte Wine dataset
print("🍷 MINI-PROJEKT: Analýza vína\n")
print("Wine dataset obsahuje chemické analýzy vín ze tří různých odrůd.")
print("\nVaše úkoly:")
print("1. [DATOVÝ VĚDEC] Prozkoumejte data a vytvořte vizualizace")
print("2. [STATISTIK] Spočítejte průměry pro každou odrůdu")
print("3. [AI INŽENÝR] Natrénujte model a zjistěte jeho přesnost")

# Načtení Wine datasetu
from sklearn.datasets import load_wine
wine = load_wine()
df_wine = pd.DataFrame(data=wine.data, columns=wine.feature_names)
df_wine['odruda'] = wine.target

print(f"\n📊 Wine dataset:")
print(f"   Počet vzorků: {len(df_wine)}")
print(f"   Počet features: {len(wine.feature_names)}")
print(f"   Počet odrůd: {df_wine['odruda'].nunique()}")

In [ ]:
# Váš kód zde - DATOVÝ VĚDEC
# Prozkoumejte data, vytvořte vizualizace

print("🔍 DATOVÝ VĚDEC: Prozkoumávám data...\n")

# Tip: Použijte df_wine.head(), df_wine.describe(), sns.pairplot()
# DOPLŇTE SVŮJ KÓD:



In [ ]:
# Váš kód zde - STATISTIK
# Spočítejte průměry, korelace

print("📐 STATISTIK: Analyzuji statistiky...\n")

# Tip: Použijte df_wine.groupby('odruda').mean(), df_wine.corr()
# DOPLŇTE SVŮJ KÓD:



In [ ]:
# Váš kód zde - AI INŽENÝR
# Natrénujte model a zjistěte přesnost

print("🤖 AI INŽENÝR: Trénuji model...\n")

# Tip: Použijte stejný postup jako pro Iris dataset
# DOPLŇTE SVŮJ KÓD:



## Shrnutí kapitoly

### Co jste se naučili:

| Role | Nástroje | Hlavní přínos |
|------|----------|---------------|
| **Datový vědec** | pandas, seaborn | Nachází příběhy v datech |
| **Statistik** | scipy, numpy | Ověřuje pravdivost poznatků |
| **AI inženýr** | sklearn | Automatizuje rozhodování |

### Klíčové poznatky:

1. **Datová věda** hledá vhled - "Co se děje v datech?"
2. **Statistika** ověřuje - "Je to opravdu tak?"
3. **AI** automatizuje - "Jak to použít pro nová data?"

### Další kroky:

- V příští kapitole se podíváme na **rozdíly mezi lidskou a strojovou inteligencí**
- Naučíte se, v čem jsou lidé stále lepší než stroje

---

*Notebook vytvořen pro kurz AI pro začátečníky*  
*Kompatibilní s Google Colab (bezplatná verze)*